In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

# 1.3a: Grids. 1.3a: 网格 (Grids)


In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from gempy.core.data import Grid
from gempy.core.data.grid_modules import RegularGrid

import gempy as gp

np.random.seed(55500)

In [ ]:
# 修复 OMP: Error #15 冲突
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

## The Grid Class Grid 类

The Grid class interacts with the rest of the data classes and grid
subclasses. Its main purpose is to feed coordinates XYZ to the
interpolator.

Grid 类与其余的数据类和网格（grid）子类进行交互。其主要目的是向插值器提供 XYZ 坐标。


In [ ]:
grid = Grid()

The most important attribute of Grid is `values` (and `values_r`
which are the values rescaled) which are the 3D points in space that
kriging will be evaluated on. This array will be fed by "grid types" on
a **composition** relation with Grid:

Grid 最重要的属性是 `values`（以及重新缩放后的值 `values_r`），它们是克里金法（kriging）将要在其上进行评估的空间 3D 点。该数组将由与 Grid 具有**组合**关系的“网格类型”提供：


<img src="../data/grids.jpg">


In [ ]:
print(grid.values)

At the moment of writing this tutorial, there are 5 grid types. The
number of grid types is scalable, and down the road we aim to connect
other grid packages (like [Discretize](https://pypi.org/project/discretize/)) as an extra Grid type.

在编写本教程时，共有 5 种网格类型。网格类型的数量是可扩展的，未来我们的目标是连接其他网格包（如 [Discretize](https://pypi.org/project/discretize/)）作为额外的 Grid 类型。


This is an enum now

现在这是一个枚举（enum）


In [ ]:
print(grid.GridTypes)

Each grid contains its own `values` attribute as well as other
methods to manipulate them depending on the type of grid.

每个网格都包含其自己的 `values` 属性以及其他根据网格类型来操作它们的方法。


In [ ]:
print(grid.values)

We can see which grids are activated (i.e. they are going to be
interpolated and therefore will live on `Grid().values`) by:

我们可以通过以下方式查看哪些网格已被激活（即它们将被插值，因此将存在于 `Grid().values` 上）：


In [ ]:
print(grid.active_grids)

By default, only the _regular grid_ (`grid.regular_grid`) is active. However, since the regular
grid is still empty, `Grid().values` is empty too.

默认情况下，只有*规则网格*（`grid.regular_grid`）是活动的。但是，由于规则网格仍然是空的，因此 `Grid().values` 也是空的。


In [ ]:
print(grid.values)

The last important attribute of Grid is the length:

Grid 的最后一个重要属性是长度（length）：


In [ ]:
print(grid.length)

Length gives back the interface indices between grids on the
`Grid().values` attribute. This can be used after interpolation to
know which interpolated values and coordinates correspond to each grid
type. You can use the method `get_grid_args` to return the indices by
name:

Length 返回 `Grid().values` 属性上各网格之间的界面索引。这可以在插值后用于了解哪些插值值和坐标对应于每种网格类型。您可以使用方法 `get_grid_args` 按名称返回索引：


In [ ]:
print(grid.topography)

By now all is a bit confusing because we have no values. Let's start
adding values to the different grids:

到目前为止，一切都有点令人困惑，因为我们没有值。让我们开始向不同的网格添加值：


### Regular Grid 规则网格 (Regular Grid)

The `Grid` class has a bunch of methods to set each grid type and
activate them.

`Grid` 类有一系列方法来设置每种网格类型并激活它们。


In [ ]:
help(RegularGrid)

In [ ]:
extent = np.array([0, 100, 0, 100, -100, 0])
resolution = np.array([20, 20, 20])
grid.dense_grid = RegularGrid(extent, resolution)
print(grid.regular_grid)  # RegularGrid will return either dense grid or octree grid depending on what is set

Now the regular grid object composed in `Grid` has been filled:

现在，`Grid` 中组合的规则网格对象已被填充：


In [ ]:
print(grid.regular_grid.values)

And the regular grid has been set active (it was already active in any
case):

并且规则网格已被设置为活动状态（无论如何它已经是活动的）：


In [ ]:
print(grid.active_grids)

Therefore, the grid values will be equal to the regular grid:

因此，网格值将等于规则网格：


In [ ]:
print(grid.values)

And the indices to extract the different arrays:

以及用于提取不同数组的索引：


In [ ]:
print(grid.length)

### Custom Grid 自定义网格 (Custom Grid)

Completely free XYZ values.

完全自由的 XYZ 值。


In [ ]:
gp.set_custom_grid(grid, np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]))

Again, `set_any_grid` will create a grid and activate it. So now the
composed object will contain values:

同样，`set_any_grid` 将创建一个网格并激活它。所以现在组合对象将包含值：


In [ ]:
print(grid.custom_grid.values)

And since it is active, it will be added to the grid.values stack:

由于它是活动的，它将被添加到 grid.values 堆栈中：


In [ ]:
print(grid.active_grids)

In [ ]:
print(grid.values.shape)

We can still recover those values with `get_grid` or by getting the
slicing args:

我们仍然可以使用 `get_grid` 或通过获取切片参数来恢复这些值：


In [ ]:
print(grid.custom_grid)

In [ ]:
print(grid.custom_grid.values)

### Topography 地形（未往下看）

Now we can set the topography. :class:`Topography <gempy.core.grid_modules.topography.Topography>`
contains methods to create manual topographies as well as using gdal for
dealing with raster data. By default, we will create a random topography:


In [ ]:
gp.set_topography_from_random(grid)

In [ ]:
print(grid.active_grids)

Now the grid values will contain both the regular grid and topography:


In [ ]:
print(grid.values, grid.length)

In [ ]:
print(grid.topography.values)

We can compare it to the topography.values:


In [ ]:
print(grid.topography.values)

Now that we have more than one grid, we can activate and deactivate any
of them in real time:


In [ ]:
grid.active_grids ^= grid.GridTypes.TOPOGRAPHY
grid.active_grids ^= grid.GridTypes.DENSE

Since now all grids are deactivated, the values will be empty:


In [ ]:
print(grid.values)

In [ ]:
grid.active_grids |= grid.GridTypes.TOPOGRAPHY

In [ ]:
print(grid.values, grid.values.shape)

In [ ]:
grid.active_grids |= grid.GridTypes.DENSE

In [ ]:
print(grid.values)

### Centered Grid

This grid contains an irregular grid where the majority of voxels are
centered around a value (or values). This type of grid is usually used
to compute certain types of forward physics where the influence
decreases with distance (e.g. gravity: Check [tutorial 2.2-Cell-selection](https://github.com/cgre-aachen/gempy/blob/master/examples/tutorials/ch2-Geophysics/ch2_2_cell_selection.py)
)


In [ ]:
gp.set_centered_grid(
    grid, centers=np.array([[300, 0, 0], [0, 0, 0]]), resolution=[10, 10, 20], radius=np.array([100, 100, 100])
)

Resolution and radius create a geometrically spaced kernel (blue dots) which
will be used to create a grid around each of the center points (red
dots):


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

ax.scatter(
    grid.centered_grid.values[:, 0], grid.centered_grid.values[:, 1], grid.centered_grid.values[:, 2], ".", alpha=0.2
)

ax.scatter(
    np.array([[300, 0, 0], [0, 0, 0]])[:, 0],
    np.array([[300, 0, 0], [0, 0, 0]])[:, 1],
    np.array([[300, 0, 0], [0, 0, 0]])[:, 2],
    c="r",
    alpha=1,
    s=30,
)

ax.set_xlim(-100, 400)
ax.set_ylim(-100, 100)
ax.set_zlim(-120, 0)
ax.set_xlabel("X Label")
ax.set_ylabel("Y Label")
ax.set_zlabel("Z Label")
plt.show()

### Section Grid

This grid type has its own tutorial. See :doc:`ch1_3b_cross_sections`
